In [1]:
#import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.filterwarnings('ignore')

In [2]:
#loading dataset
df=pd.read_csv("iris.csv")

In [3]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [5]:
df.drop('Id', axis=1, inplace=True)

In [6]:
df['Species']=df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica':2})

In [7]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [8]:
df['Species'].value_counts()

0    50
1    50
2    50
Name: Species, dtype: int64

### Model Deployment

In [9]:
X=df.drop("Species", axis=1)
y=df['Species']

In [10]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=400)

In [11]:
dummy=DummyClassifier()
dummy.fit(X_train, y_train)
pred=dummy.predict(X_test)
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [12]:
(y_test==pred).mean()

0.3

In [13]:
accuracy_score(y_test, pred)

0.3

In [14]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.30      1.00      0.46         9
           2       0.00      0.00      0.00        10

    accuracy                           0.30        30
   macro avg       0.10      0.33      0.15        30
weighted avg       0.09      0.30      0.14        30



In [15]:
log=LogisticRegression()
log.fit(X_train, y_train)
log_pred=log.predict(X_test)


In [16]:
print(classification_report(y_test, log_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [17]:
import pickle

# saving the model in a pickle file
model = pickle.dump(log, open("model.pkl","wb"))

# loading the model int the console
model = pickle.load(open('model.pkl','rb'))



In [18]:
print(model.predict([[4.7,3.2,1.3,0.2]]))

[0]


In [ ]:
import numpy as np
from flask import Flask, request, render_template
import pickle

#Create an app object using the Flask class. 
app = Flask(__name__)

#Load the trained model. (Pickle file)
model = pickle.load(open('model.pkl', 'rb'))


@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    int_features = [float(x) for x in request.form.values()]
    features = [np.array(int_features)]
    prediction = model.predict(features)
    
    output = prediction[0]
    
    if output == 0:
        prediction_text = 'Iris-setosa'
    elif output == 1:
        prediction_text = 'Iris-versicolor'
    else:
        prediction_text = 'Iris-virginica'
    
    return render_template('index.html', prediction_text=prediction_text)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Mar/2024 12:44:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 12:45:06] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 12:46:37] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 14:27:27] "POST /predict HTTP/1.1" 200 -
